In [222]:
#Importing dependencies...not sure if I'll be using all of these, but helps to be prepared :)

import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
import json
from geopy.geocoders import Nominatim
from pandas.io.json import json_normalize
import requests
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
import folium


In [223]:
#Importing the Dataset from zipdatamaps.com for zip code analysis.
df = pd.read_html("https://www.zipdatamaps.com/san-francisco-ca-county-zipcodes")[1]
#Looking at the table on the website is probably sufficient, but note here that only 26 zip codes
#have a population value. The rest will be dropped.
df

List of All Zipcodes in San Francisco County, California                 \
                                                   ZIP Code  ZIP Code Name   
0                                             94102.0        San Francisco   
1                                             94103.0        San Francisco   
2                                             94104.0        San Francisco   
3                                             94105.0        San Francisco   
4                                             94107.0        San Francisco   
5                                                 NaN                  NaN   
6                                             94108.0        San Francisco   
7                                             94109.0        San Francisco   
8                                             94110.0        San Francisco   
9                                             94111.0        San Francisco   
10                                            94112.0        San Francisco   
11                                            94114.0        San Francisco   
12                                            94115.0        San Francisco   
13                                            94116.0        San Francisco   
14                                            94117.0        San Francisco   
15                                            94118.0        San Francisco   
16                                            94121.0        San Francisco   
17                                            94122.0        San Francisco   
18                                            94123.0        San Francisco   
19                                            94124.0        San Francisco   
20                                            94127.0        San Francisco   
21                                            94129.0        San Francisco   
22                                            94130.0        San Francisco   
23                                            94131.0        San Francisco   
24                                            94132.0        San Francisco   
25                                            94133.0        San Francisco   
26                                            94134.0        San Francisco   
27                                            94158.0        San Francisco   
28                                            94188.0        San Francisco   
29                                            94119.0        San Francisco   
30                                            94120.0        San Francisco   
31                                            94125.0        San Francisco   
32                                            94126.0        San Francisco   
33                                            94140.0        San Francisco   
34                                            94141.0        San Francisco   
35                                            94142.0        San Francisco   
36                                            94146.0        San Francisco   
37                                            94147.0        San Francisco   
38                                            94159.0        San Francisco   
39                                            94164.0        San Francisco   
40                                            94172.0        San Francisco   
41                                            94137.0        San Francisco   
42                                            94139.0        San Francisco   
43                                            94143.0        San Francisco   
44                                            94144.0        San Francisco   
45                                            94145.0        San Francisco   
46                                            94151.0        San Francisco   
47                                            94153.0        San Francisco   
48                                            94154.0        San Francisco   
49                       

In [320]:
#The original has this "List of all zip codes..." in EVERY column name, which needs to be removed for convenience.
df.columns = df.columns.droplevel()

In [225]:
#Filtering out the Unique values for ZIP Type, as they do not have population numbers associated with them.
df = df[df['ZIP Type'] == "Non-Unique"]

df

,ZIP Code,ZIP Code Name,Population,ZIP Type
0,94102.0,San Francisco,31176.0,Non-Unique
1,94103.0,San Francisco,27170.0,Non-Unique
2,94104.0,San Francisco,406.0,Non-Unique
3,94105.0,San Francisco,5846.0,Non-Unique
4,94107.0,San Francisco,26599.0,Non-Unique
6,94108.0,San Francisco,13768.0,Non-Unique
7,94109.0,San Francisco,55984.0,Non-Unique
8,94110.0,San Francisco,69333.0,Non-Unique
9,94111.0,San Francisco,3713.0,Non-Unique
10,94112.0,San Francisco,79407.0,Non-Unique


In [226]:
#One NaN value persists in this case, I think we can be lazy here and just drop the last row.
df = df[:-1]

#Converting the float values to int
df['ZIP Code'] = df['ZIP Code'].astype('int')
df['Population'] = df['Population'].astype('int')

#Don't need the ZIP Type colum anymore...or ZIP Code Name for that matter.
del df['ZIP Code Name']
del df['ZIP Type']

df

<ipython-input-226-657f3e5e1258>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['ZIP Code'] = df['ZIP Code'].astype('int')
<ipython-input-226-657f3e5e1258>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Population'] = df['Population'].astype('int')


,ZIP Code,Population
0,94102,31176
1,94103,27170
2,94104,406
3,94105,5846
4,94107,26599
6,94108,13768
7,94109,55984
8,94110,69333
9,94111,3713
10,94112,79407


In [230]:
df.rename(columns = {"ZIP Code" : "Neighborhood"}, inplace = True)
df.head()

,Neighborhood,Population
0,94102,31176
1,94103,27170
2,94104,406
3,94105,5846
4,94107,26599


In [232]:
#Turns out ZIP Data Maps has Lat and Lon coordinates, so we'll just get them now.
df = df.reset_index()
lister = df['Neighborhood']
coordlist = []

for i in range(0,len(lister)):
    info = pd.read_html("http://www.zipdatamaps.com/" + str(lister[i]))[0]
    coord = info[1].values[-1]
    coordlist.append(coord)

In [233]:
#Adding the coordinates to the DataFrame

lat = []
lon = []
for i in range(0, len(coordlist)):
    test = coordlist[i].split(", ")
    lat.append(test[0])
    lon.append(test[1])
    
df['Latitude'] = lat
df['Longitude'] = lon

df.head()

,index,Neighborhood,Population,Latitude,Longitude
0,0,94102,31176,37.77938843,-122.41828156
1,1,94103,27170,37.77603912,-122.41335297
2,2,94104,406,37.79093170,-122.40177917
3,3,94105,5846,37.78956985,-122.39475250
4,4,94107,26599,37.76816177,-122.39868164


In [234]:
#Time to analyze
#Foursquare Credentials
CLIENT_ID = 'ZJHMOFZFGGWGUXYX5S4UWONHKRFGL4KB3U33L3OUTYLCMYI4' # your Foursquare ID
CLIENT_SECRET = '2STKYCGH2GN0JIZ1IPHJYEBQGLVYFMNK2K5SLATYHQHPYXOF' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

#Function from Week 3 exercise
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

Your credentails:
CLIENT_ID: ZJHMOFZFGGWGUXYX5S4UWONHKRFGL4KB3U33L3OUTYLCMYI4
CLIENT_SECRET:2STKYCGH2GN0JIZ1IPHJYEBQGLVYFMNK2K5SLATYHQHPYXOF


In [236]:
LIMIT = 100
radius = 500
sf_venues = getNearbyVenues(names=df['Neighborhood'],
                                   latitudes=df['Latitude'],
                                   longitudes=df['Longitude']
                                  )

94102
94103
94104
94105
94107
94108
94109
94110
94111
94112
94114
94115
94116
94117
94118
94121
94122
94123
94124
94127
94129
94130
94131
94132
94133
94134
94158


In [237]:
sf_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,94102,37.77938843,-122.41828156,Asian Art Museum,37.780178,-122.416505,Art Museum
1,94102,37.77938843,-122.41828156,Louise M. Davies Symphony Hall,37.777976,-122.420157,Concert Hall
2,94102,37.77938843,-122.41828156,Herbst Theater,37.779548,-122.420953,Concert Hall
3,94102,37.77938843,-122.41828156,War Memorial Opera House,37.778601,-122.420816,Opera House
4,94102,37.77938843,-122.41828156,San Francisco Ballet,37.778580,-122.420798,Dance Studio


In [238]:
print(sf_venues.shape)
sf_venues.head()

(1326, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,94102,37.77938843,-122.41828156,Asian Art Museum,37.780178,-122.416505,Art Museum
1,94102,37.77938843,-122.41828156,Louise M. Davies Symphony Hall,37.777976,-122.420157,Concert Hall
2,94102,37.77938843,-122.41828156,Herbst Theater,37.779548,-122.420953,Concert Hall
3,94102,37.77938843,-122.41828156,War Memorial Opera House,37.778601,-122.420816,Opera House
4,94102,37.77938843,-122.41828156,San Francisco Ballet,37.778580,-122.420798,Dance Studio


In [239]:
sf_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
94102,84,84,84,84,84,84
94103,90,90,90,90,90,90
94104,100,100,100,100,100,100
94105,82,82,82,82,82,82
94107,59,59,59,59,59,59
94108,93,93,93,93,93,93
94109,64,64,64,64,64,64
94110,42,42,42,42,42,42
94111,71,71,71,71,71,71


In [240]:
print('There are {} unique categories.'.format(len(sf_venues['Venue Category'].unique())))

There are 261 unique categories.


In [241]:
sf_onehot = pd.get_dummies(sf_venues[['Venue Category']], prefix="", prefix_sep="")


sf_onehot['Neighborhood'] = sf_venues['Neighborhood'] 


fixed_columns = [sf_onehot.columns[-1]] + list(sf_onehot.columns[:-1])
sf_onehot = sf_onehot[fixed_columns]

sf_onehot.head()

,Yoga Studio,ATM,Accessories Store,Adult Boutique,African Restaurant,American Restaurant,Antique Shop,Argentinian Restaurant,Art Gallery,Art Museum,...,Turkish Restaurant,Tuscan Restaurant,Udon Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Whisky Bar,Wine Bar,Wine Shop,Wings Joint,Women's Store
0,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [242]:
sf_onehot.shape

(1326, 261)

In [243]:
sf_grouped = sf_onehot.groupby('Neighborhood').mean().reset_index()
sf_grouped

,Neighborhood,Yoga Studio,ATM,Accessories Store,Adult Boutique,African Restaurant,American Restaurant,Antique Shop,Argentinian Restaurant,Art Gallery,...,Turkish Restaurant,Tuscan Restaurant,Udon Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Whisky Bar,Wine Bar,Wine Shop,Wings Joint,Women's Store
0,94102,0.000000,0.000000,0.000000,0.00,0.000000,0.011905,0.000000,0.000000,0.000000,...,0.00,0.000000,0.00,0.035714,0.035714,0.000000,0.011905,0.011905,0.000000,0.000000
1,94103,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.055556,...,0.00,0.000000,0.00,0.011111,0.000000,0.000000,0.033333,0.000000,0.000000,0.000000
2,94104,0.000000,0.000000,0.000000,0.00,0.000000,0.010000,0.000000,0.000000,0.000000,...,0.00,0.000000,0.00,0.000000,0.000000,0.010000,0.000000,0.010000,0.000000,0.000000
3,94105,0.012195,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.036585,...,0.00,0.000000,0.00,0.000000,0.012195,0.000000,0.000000,0.000000,0.000000,0.000000
4,94107,0.016949,0.016949,0.000000,0.00,0.000000,0.050847,0.016949,0.016949,0.033898,...,0.00,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.016949,0.000000,0.000000
5,94108,0.010753,0.000000,0.000000,0.00,0.000000,0.021505,0.000000,0.000000,0.000000,...,0.00,0.000000,0.00,0.010753,0.021505,0.010753,0.000000,0.000000,0.000000,0.000000
6,94109,0.000000,0.000000,0.000000,0.00,0.015625,0.031250,0.000000,0.000000,0.000000,...,0.00,0.000000,0.00,0.000000,0.015625,0.000000,0.046875,0.031250,0.015625,0.000000
7,94110,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.023810,...,0.00,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
8,94111,0.000000,0.000000,0.000000,0.00,0.000000,0.042254,0.000000,0.000000,0.000000,...,0.00,0.000000,0.00,0.000000,0.014085,0.014085,0.014085,0.000000,0.000000,0.000000
9,94112,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [244]:
sf_grouped.shape

(26, 261)

In [245]:
num_top_venues = 5

for hood in sf_grouped['Neighborhood']:
    print("----"+str(hood)+"----")
    temp = sf_grouped[sf_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')



----94102----
                   venue  freq
0                   Café  0.06
1            Coffee Shop  0.06
2                 Bakery  0.04
3                  Hotel  0.04
4  Vietnamese Restaurant  0.04


----94103----
          venue  freq
0   Coffee Shop  0.08
1   Art Gallery  0.06
2  Cocktail Bar  0.06
3       Gay Bar  0.04
4     Nightclub  0.04


----94104----
                 venue  freq
0          Coffee Shop  0.07
1          Salad Place  0.04
2       Sandwich Place  0.04
3  Japanese Restaurant  0.04
4         Optical Shop  0.03


----94105----
                  venue  freq
0           Coffee Shop  0.16
1                  Café  0.06
2                   Gym  0.06
3            Food Truck  0.05
4  Gym / Fitness Center  0.05


----94107----
                    venue  freq
0  Furniture / Home Store  0.07
1                    Park  0.07
2                     Gym  0.05
3     American Restaurant  0.05
4             Coffee Shop  0.05


----94108----
            venue  freq
0           Hotel 

In [246]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [300]:
num_top_venues = 20

indicators = ['st', 'nd', 'rd']


columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))


neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = sf_grouped['Neighborhood']

for ind in np.arange(sf_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(sf_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,...,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue,16th Most Common Venue,17th Most Common Venue,18th Most Common Venue,19th Most Common Venue,20th Most Common Venue
0,94102,Café,Coffee Shop,Hotel,Bakery,Vietnamese Restaurant,Vegetarian / Vegan Restaurant,Theater,Concert Hall,French Restaurant,...,Beer Bar,Pizza Place,Sushi Restaurant,Poke Place,Cocktail Bar,Sandwich Place,Opera House,Pharmacy,Credit Union,Ramen Restaurant
1,94103,Coffee Shop,Cocktail Bar,Art Gallery,Nightclub,Gay Bar,Restaurant,Sushi Restaurant,Furniture / Home Store,Café,...,Motorcycle Shop,Clothing Store,Beer Bar,Bar,Gym,Bakery,Theater,Thai Restaurant,Dance Studio,Deli / Bodega
2,94104,Coffee Shop,Salad Place,Japanese Restaurant,Sandwich Place,Gym,Boutique,Optical Shop,Food Truck,Juice Bar,...,Seafood Restaurant,Restaurant,Clothing Store,Cocktail Bar,Convenience Store,Men's Store,Sushi Restaurant,Lounge,Greek Restaurant,Hotel
3,94105,Coffee Shop,Café,Gym,Food Truck,Gym / Fitness Center,Lounge,Art Gallery,Seafood Restaurant,Liquor Store,...,Italian Restaurant,Dim Sum Restaurant,Park,Cocktail Bar,Bus Station,Salad Place,Sandwich Place,Market,Outdoor Sculpture,Office
4,94107,Park,Furniture / Home Store,Gym,Coffee Shop,American Restaurant,Art Gallery,Japanese Restaurant,Arts & Crafts Store,Chinese Restaurant,...,Mexican Restaurant,Café,Food Truck,Mediterranean Restaurant,Burger Joint,Bubble Tea Shop,Rock Club,Yoga Studio,Food Stand,Ramen Restaurant


In [302]:


# set number of clusters
kclusters = 8

sf_grouped_clustering = sf_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters).fit(sf_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]


array([2, 2, 2, 1, 2, 2, 2, 2, 2, 6])

In [303]:
# add clustering labels
#del neighborhoods_venues_sorted
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

sf_merged = df

sf_merged = sf_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood',how='right')

sf_merged.head() # check the last columns!

,index,Neighborhood,Population,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,...,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue,16th Most Common Venue,17th Most Common Venue,18th Most Common Venue,19th Most Common Venue,20th Most Common Venue
0,0,94102,31176,37.77938843,-122.41828156,2,Café,Coffee Shop,Hotel,Bakery,...,Beer Bar,Pizza Place,Sushi Restaurant,Poke Place,Cocktail Bar,Sandwich Place,Opera House,Pharmacy,Credit Union,Ramen Restaurant
1,1,94103,27170,37.77603912,-122.41335297,2,Coffee Shop,Cocktail Bar,Art Gallery,Nightclub,...,Motorcycle Shop,Clothing Store,Beer Bar,Bar,Gym,Bakery,Theater,Thai Restaurant,Dance Studio,Deli / Bodega
2,2,94104,406,37.79093170,-122.40177917,2,Coffee Shop,Salad Place,Japanese Restaurant,Sandwich Place,...,Seafood Restaurant,Restaurant,Clothing Store,Cocktail Bar,Convenience Store,Men's Store,Sushi Restaurant,Lounge,Greek Restaurant,Hotel
3,3,94105,5846,37.78956985,-122.39475250,1,Coffee Shop,Café,Gym,Food Truck,...,Italian Restaurant,Dim Sum Restaurant,Park,Cocktail Bar,Bus Station,Salad Place,Sandwich Place,Market,Outdoor Sculpture,Office
4,4,94107,26599,37.76816177,-122.39868164,2,Park,Furniture / Home Store,Gym,Coffee Shop,...,Mexican Restaurant,Café,Food Truck,Mediterranean Restaurant,Burger Joint,Bubble Tea Shop,Rock Club,Yoga Studio,Food Stand,Ramen Restaurant


In [304]:
neighborhoods_venues_sorted

,Cluster Labels,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,...,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue,16th Most Common Venue,17th Most Common Venue,18th Most Common Venue,19th Most Common Venue,20th Most Common Venue
0,2,94102,Café,Coffee Shop,Hotel,Bakery,Vietnamese Restaurant,Vegetarian / Vegan Restaurant,Theater,Concert Hall,...,Beer Bar,Pizza Place,Sushi Restaurant,Poke Place,Cocktail Bar,Sandwich Place,Opera House,Pharmacy,Credit Union,Ramen Restaurant
1,2,94103,Coffee Shop,Cocktail Bar,Art Gallery,Nightclub,Gay Bar,Restaurant,Sushi Restaurant,Furniture / Home Store,...,Motorcycle Shop,Clothing Store,Beer Bar,Bar,Gym,Bakery,Theater,Thai Restaurant,Dance Studio,Deli / Bodega
2,2,94104,Coffee Shop,Salad Place,Japanese Restaurant,Sandwich Place,Gym,Boutique,Optical Shop,Food Truck,...,Seafood Restaurant,Restaurant,Clothing Store,Cocktail Bar,Convenience Store,Men's Store,Sushi Restaurant,Lounge,Greek Restaurant,Hotel
3,1,94105,Coffee Shop,Café,Gym,Food Truck,Gym / Fitness Center,Lounge,Art Gallery,Seafood Restaurant,...,Italian Restaurant,Dim Sum Restaurant,Park,Cocktail Bar,Bus Station,Salad Place,Sandwich Place,Market,Outdoor Sculpture,Office
4,2,94107,Park,Furniture / Home Store,Gym,Coffee Shop,American Restaurant,Art Gallery,Japanese Restaurant,Arts & Crafts Store,...,Mexican Restaurant,Café,Food Truck,Mediterranean Restaurant,Burger Joint,Bubble Tea Shop,Rock Club,Yoga Studio,Food Stand,Ramen Restaurant
5,2,94108,Hotel,Coffee Shop,Boutique,Cocktail Bar,Clothing Store,Spa,Japanese Restaurant,Kitchen Supply Store,...,Sushi Restaurant,Gym,Bar,Jewelry Store,French Restaurant,Bubble Tea Shop,Church,American Restaurant,Italian Restaurant,Electronics Store
6,2,94109,Bar,Wine Bar,Gym / Fitness Center,Sushi Restaurant,Coffee Shop,Italian Restaurant,Pet Store,Wine Shop,...,American Restaurant,Gym,Pizza Place,Bus Stop,Ice Cream Shop,Garden,Café,Smoothie Shop,Boutique,Bookstore
7,2,94110,Mexican Restaurant,Coffee Shop,Dive Bar,Grocery Store,Pizza Place,Gym / Fitness Center,Park,Food & Drink Shop,...,Massage Studio,South American Restaurant,Burrito Place,Café,Cocktail Bar,Fish Market,Restaurant,New American Restaurant,Event Space,Pool
8,2,94111,Food Truck,Café,Pier,American Restaurant,Italian Restaurant,Street Food Gathering,Bar,Coffee Shop,...,Harbor / Marina,Shipping Store,Seafood Restaurant,Clothing Store,New American Restaurant,Scenic Lookout,Science Museum,Bookstore,Mexican Restaurant,Spanish Restaurant
9,6,94112,Bus Station,Dessert Shop,Park,Metro Station,Flower Shop,Baseball Field,BBQ Joint,Light Rail Station,...,Photography Studio,Ethiopian Restaurant,Empanada Restaurant,Doctor's Office,Garden,Furniture / Home Store,Frozen Yogurt Shop,Fried Chicken Joint,French Restaurant,Fountain


In [305]:
address = 'San Francisco'

geolocator = Nominatim(user_agent="sf_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of San Francisco are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of San Francisco are 37.7790262, -122.4199061.


In [306]:
sf_merged

,index,Neighborhood,Population,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,...,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue,16th Most Common Venue,17th Most Common Venue,18th Most Common Venue,19th Most Common Venue,20th Most Common Venue
0,0,94102,31176,37.77938843,-122.41828156,2,Café,Coffee Shop,Hotel,Bakery,...,Beer Bar,Pizza Place,Sushi Restaurant,Poke Place,Cocktail Bar,Sandwich Place,Opera House,Pharmacy,Credit Union,Ramen Restaurant
1,1,94103,27170,37.77603912,-122.41335297,2,Coffee Shop,Cocktail Bar,Art Gallery,Nightclub,...,Motorcycle Shop,Clothing Store,Beer Bar,Bar,Gym,Bakery,Theater,Thai Restaurant,Dance Studio,Deli / Bodega
2,2,94104,406,37.79093170,-122.40177917,2,Coffee Shop,Salad Place,Japanese Restaurant,Sandwich Place,...,Seafood Restaurant,Restaurant,Clothing Store,Cocktail Bar,Convenience Store,Men's Store,Sushi Restaurant,Lounge,Greek Restaurant,Hotel
3,3,94105,5846,37.78956985,-122.39475250,1,Coffee Shop,Café,Gym,Food Truck,...,Italian Restaurant,Dim Sum Restaurant,Park,Cocktail Bar,Bus Station,Salad Place,Sandwich Place,Market,Outdoor Sculpture,Office
4,4,94107,26599,37.76816177,-122.39868164,2,Park,Furniture / Home Store,Gym,Coffee Shop,...,Mexican Restaurant,Café,Food Truck,Mediterranean Restaurant,Burger Joint,Bubble Tea Shop,Rock Club,Yoga Studio,Food Stand,Ramen Restaurant
5,6,94108,13768,37.79093170,-122.40892029,2,Hotel,Coffee Shop,Boutique,Cocktail Bar,...,Sushi Restaurant,Gym,Bar,Jewelry Store,French Restaurant,Bubble Tea Shop,Church,American Restaurant,Italian Restaurant,Electronics Store
6,7,94109,55984,37.79748917,-122.42076874,2,Bar,Wine Bar,Gym / Fitness Center,Sushi Restaurant,...,American Restaurant,Gym,Pizza Place,Bus Stop,Ice Cream Shop,Garden,Café,Smoothie Shop,Boutique,Bookstore
7,8,94110,69333,37.74868011,-122.41504669,2,Mexican Restaurant,Coffee Shop,Dive Bar,Grocery Store,...,Massage Studio,South American Restaurant,Burrito Place,Café,Cocktail Bar,Fish Market,Restaurant,New American Restaurant,Event Space,Pool
8,9,94111,3713,37.79978180,-122.39974213,2,Food Truck,Café,Pier,American Restaurant,...,Harbor / Marina,Shipping Store,Seafood Restaurant,Clothing Store,New American Restaurant,Scenic Lookout,Science Museum,Bookstore,Mexican Restaurant,Spanish Restaurant
9,10,94112,79407,37.72204971,-122.44474030,6,Bus Station,Dessert Shop,Park,Metro Station,...,Photography Studio,Ethiopian Restaurant,Empanada Restaurant,Doctor's Office,Garden,Furniture / Home Store,Frozen Yogurt Shop,Fried Chicken Joint,French Restaurant,Fountain


In [318]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(sf_merged['Latitude'], sf_merged['Longitude'], sf_merged['Neighborhood'], sf_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [317]:
sf_merged.loc[sf_merged['Cluster Labels'] == 2, sf_merged.columns[[1] + list(range(5, sf_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,...,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue,16th Most Common Venue,17th Most Common Venue,18th Most Common Venue,19th Most Common Venue,20th Most Common Venue
0,94102,2,Café,Coffee Shop,Hotel,Bakery,Vietnamese Restaurant,Vegetarian / Vegan Restaurant,Theater,Concert Hall,...,Beer Bar,Pizza Place,Sushi Restaurant,Poke Place,Cocktail Bar,Sandwich Place,Opera House,Pharmacy,Credit Union,Ramen Restaurant
1,94103,2,Coffee Shop,Cocktail Bar,Art Gallery,Nightclub,Gay Bar,Restaurant,Sushi Restaurant,Furniture / Home Store,...,Motorcycle Shop,Clothing Store,Beer Bar,Bar,Gym,Bakery,Theater,Thai Restaurant,Dance Studio,Deli / Bodega
2,94104,2,Coffee Shop,Salad Place,Japanese Restaurant,Sandwich Place,Gym,Boutique,Optical Shop,Food Truck,...,Seafood Restaurant,Restaurant,Clothing Store,Cocktail Bar,Convenience Store,Men's Store,Sushi Restaurant,Lounge,Greek Restaurant,Hotel
4,94107,2,Park,Furniture / Home Store,Gym,Coffee Shop,American Restaurant,Art Gallery,Japanese Restaurant,Arts & Crafts Store,...,Mexican Restaurant,Café,Food Truck,Mediterranean Restaurant,Burger Joint,Bubble Tea Shop,Rock Club,Yoga Studio,Food Stand,Ramen Restaurant
5,94108,2,Hotel,Coffee Shop,Boutique,Cocktail Bar,Clothing Store,Spa,Japanese Restaurant,Kitchen Supply Store,...,Sushi Restaurant,Gym,Bar,Jewelry Store,French Restaurant,Bubble Tea Shop,Church,American Restaurant,Italian Restaurant,Electronics Store
6,94109,2,Bar,Wine Bar,Gym / Fitness Center,Sushi Restaurant,Coffee Shop,Italian Restaurant,Pet Store,Wine Shop,...,American Restaurant,Gym,Pizza Place,Bus Stop,Ice Cream Shop,Garden,Café,Smoothie Shop,Boutique,Bookstore
7,94110,2,Mexican Restaurant,Coffee Shop,Dive Bar,Grocery Store,Pizza Place,Gym / Fitness Center,Park,Food & Drink Shop,...,Massage Studio,South American Restaurant,Burrito Place,Café,Cocktail Bar,Fish Market,Restaurant,New American Restaurant,Event Space,Pool
8,94111,2,Food Truck,Café,Pier,American Restaurant,Italian Restaurant,Street Food Gathering,Bar,Coffee Shop,...,Harbor / Marina,Shipping Store,Seafood Restaurant,Clothing Store,New American Restaurant,Scenic Lookout,Science Museum,Bookstore,Mexican Restaurant,Spanish Restaurant
10,94114,2,Gay Bar,Yoga Studio,Coffee Shop,Grocery Store,Park,Thai Restaurant,Convenience Store,Playground,...,Pharmacy,Pilates Studio,Pet Store,Optical Shop,New American Restaurant,Seafood Restaurant,Nail Salon,Japanese Restaurant,Indian Restaurant,Hill
11,94115,2,Bakery,Spa,American Restaurant,Ice Cream Shop,Pizza Place,Tea Room,Boutique,Chinese Restaurant,...,Gym / Fitness Center,Italian Restaurant,Sushi Restaurant,Cosmetics Shop,Yoga Studio,New American Restaurant,Wine Bar,Arts & Crafts Store,Vietnamese Restaurant,Park
